In [1]:
import pandas as pd
df = pd.read_csv('airpass.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [2]:
df.isnull().sum().sum()

310

In [3]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

In [4]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: count, dtype: float64

In [5]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: count, dtype: float64

In [6]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: count, dtype: float64

In [7]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [8]:
df=pd.get_dummies(df)
df.shape

(103904, 27)

In [14]:
df = df.drop('Unnamed: 0', axis = 1)

In [15]:
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,Class_Business,Class_Eco,Class_Eco Plus
0,70172,0,1,13,0,460,3,4,3,1,...,4,4,5,5,25,18.0,0,False,False,True
1,5047,0,0,25,1,235,3,2,3,3,...,3,1,4,1,1,6.0,0,True,False,False
2,110028,1,1,26,1,1142,2,2,2,2,...,4,4,4,5,0,0.0,1,True,False,False
3,24026,1,1,25,1,562,2,5,5,5,...,3,1,4,2,11,9.0,0,True,False,False
4,119299,0,1,61,1,214,3,3,3,3,...,4,3,3,3,0,0.0,1,True,False,False


In [17]:
from sklearn.model_selection import train_test_split

X = df.drop('satisfaction', axis = 1)
y = df['satisfaction']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [18]:
# для нормализации, стандартизации
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [21]:
from sklearn import metrics
from sklearn import linear_model
model_lr = linear_model.LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.8547046934650797

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(metrics.f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.9s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.5s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.5s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.5s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   1.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.9s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   1.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.7s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.8s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [30]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 MB 1.2 MB/s eta 0:00:0000:0100:02m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 239.8 kB/s eta 0:00:00 0:00:01
  error: subprocess-exited-with-error
  
  × Building wheel for catboost (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.9-universal2-cpython-39/catboost
      copying catboost/monoforest.py -> build/lib.macosx-10.9-universal2-cpython-39/catboost
      copying catboost/plot_helpers.py -> build/lib.macosx-10.9-universal2-cpython-39/catboost
      copying catboost/metrics.py -> build/lib.macosx-10.9-universal2-cpython-39/catboost
      copying catboost/carry.py -> build/lib.macosx-10.9-universal2-cpython-39/catboost
      copying catboost/

In [31]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

ModuleNotFoundError: No module named 'catboost'

In [ ]:
cm = get_confusion_matrix(model, Pool(X_train, y_train))

In [ ]:
pd.DataFrame(
    {
        "feature_importance": clf.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)